# Item 40: Initialize Parent Classes with `super`

In [1]:
# The old, simple way to initialize a parent class from a child class is to directly call the parent class's
# __init__ method with the child instance
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

This approach works fine for simple class hierarchies but breaks in many cases. If a class is affected by multiple inheritance (something to avoid in general), calling the superclasses' `__init__` methods directly can lead to unpredictable behavior.

One problem is that the `__init__` call order isn't specified across all subclasses.

In [2]:
# For example, here we define two parent classes that operate on the instance's value field
class TimesTwo:
    def __init__(self):
        self.value *= 2
    
class PlusFive:
    def __init__(self):
        self.value += 5

In [3]:
# This class defines its parent classes in one ordering
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [4]:
# And constructing it produces a result that matches the parent class ordering
foo = OneWay(5)
print('First ordering value is (5 * 2) + 5 =', foo.value)

First ordering value is (5 * 2) + 5 = 15


In [6]:
# Here's a parent class that defines the same parent classes but in a different ordering. However, we left the
# calls to the parent class constructors in the same order as before, which means that this class's behavior 
# doesn't match the order of the parent classes in its definition.
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [7]:
bar = AnotherWay(5)
print('Second ordering value is', bar.value)

Second ordering value is 15


Another problem occurs with diamond inheritance, which happens when a subclass inherits from two separate classes that have the same suplerclass somewhere in the hierarchy. Diamond inheritance causes the common superclass's `__init__` method to run multiple times, causing unexpected behavior.

In [8]:
# Here we define two child classes that inherit from MyBaseClass
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7

class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

In [9]:
# Then, we define a child class that inherits from both of the previous classes, making MyBaseClass the top of
# the diamond
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)

foo = ThisWay(5)
print('Should be (5 * 7) + 9 = 44 but is', foo.value)

Should be (5 * 7) + 9 = 44 but is 14


The call to the second parent's class constructor, `PlusNine.__init__`, causes `self.value` to be reset back to 5 when `MyBaseClass.__init__` gets called a second time. That results in the calculation of `self.value` to be 5 + 9 = 14, completely ignoring the effect of `TimesSevem.__init__` constructor. This behavior is surprising and can be very difficult to debug.

To solve these problems, Python has the `super` built-in function and standard method resolution order (MRO). `super` ensures that common superclasses in diamond hierarchies are run only once. The MRO defines the ordering in which superclasses are initialized.

In [10]:
# Here we create a diamond-shaped class hierarchy again, but this time using super
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7


class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

In [11]:
# Now, the top part of the diamond, MyBaseClass.__init__, is run only once. The other parent classes are run
# in the order specified in the class statement
class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

foo = GoodWay(5)
print('Should be 7 * (5 + 9) = 98 abd is', foo.value)

Should be 7 * (5 + 9) = 98 abd is 98


The order may seem backward in the previous snippet at first. Shouldn't `TimesSevenCorrect.__init__` have run first? Shouldn't the result be (5 * 9) + 9 = 44? The answer is **no**. This ordering mathces what the MRO defines for this class.